# Tema 2.4: Métodos Especiales


## 1. Métodos Especiales

Python permite definir comportamientos especiales para los objetos mediante métodos que empiezan y terminan con doble guion bajo (`__metodo__`), llamados *Dunder Methods*.

Su propósito es permitir que estos objetos puedan utilizarse en operaciones aritméticas, de pertenencia, de comparación, etc.

Algunos ejemplos:

*   `__str__(self)`: 
    + Representación en cadena de texto amigable y legible (para poder hacer un `print()` o `str()` sobre el objeto).
    + P.e. `print(x)` invoca `print(x.__str__())`
    + Este método se implementa por defecto como `return self.__repr__()`, lo que invoca al método `__repr__()`.

*   `__repr__(self)`: 
    + Representación técnica (para depuración, con `repr()` o en consola interactiva).
    + P.e. `repr(x)` invoca `x.__repr__()`
    + Este método se implementa por defecto mostrando el nombre del módulo, el nombre de la clase y la dirección de memoria hexadecimal del objeto
      + P.e. `<__main__.Persona object at 0x7f8e1c1b2d10>`.

*   `__eq__(self, other)`: 
    + Define la operación de igualdad (`==`) entre el propio objeto `self` y otro objeto `other` de la misma clase.
    + P.e. `x == y` invoca `x.__eq__(y)`
    + Este método se implementa por defecto como `return self is other`, lo que compara la identidad de los objetos 
      + Es decir, de forma equivalente, comprueba si `id(self) == id(other)`.

*   `__add__(self, other)`: 
    + Define la suma (`+`) de.
    + P.e. `x + y` invoca `x.__add__(y)`

*   `__mul__(self, other)`:
    + Define la multiplicación (`*`).
    + P.e. `x * y` invoca `x.__mul__(y)`.

*   `__lt__(self, other)`:
    + Define la operación menor que (`<`).
    + P.e. `x < y` invoca `x.__lt__(y)`.

*   `__len__(self)`:
    + Define el cálculo del tamaño (`len()`) del objeto.
    + P.e. `len(x)` invoca `x.__len__()`.

*   `__getitem__(self, key)`:
    + Permite acceder a los elementos del objeto usando corchetes (`[]`).
    + P.e. `x[key]` invoca `x.__getitem__(key)`.

*   `__setitem__(self, key, value)`:
    + Permite asignar valores a los elementos del objeto usando corchetes (`[]`).
    + P.e. `x[key] = value` invoca `x.__setitem__(key, value)`.


Para más información, consultar la sección ["Métodos especiales" del Seminario de Python3](https://dsic.gitbook.io/python3/clases/metodos-especiales).



In [1]:
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}, {self.y})"

    def __eq__(self, otro):
        return self.x == otro.x and self.y == otro.y

    def __add__(self, otro):
        return Vector2D(self.x + otro.x, self.y + otro.y)

    def __mul__(self, escalar):
        return Vector2D(self.x * escalar, self.y * escalar)

    def __len__(self):
        return 2  # Un Vector2D siempre tiene 2 componentes

    def __lt__(self, otro):
        # Comparación por magnitud geométrica (cuadrática para simplificar raíz)
        return (self.x**2 + self.y**2) < (otro.x**2 + otro.y**2)

    def __getitem__(self, indice):
        if indice == 0:
            return self.x
        elif indice == 1:
            return self.y
        else:
            raise IndexError("El Vector2D solo tiene índices 0 (x) y 1 (y)")

    def __setitem__(self, indice, valor):
        if indice == 0:
            self.x = valor
        elif indice == 1:
            self.y = valor
        else:
            raise IndexError("El Vector2D solo tiene índices 0 (x) y 1 (y)")

v1 = Vector2D(2, 3)
v2 = Vector2D(1, 4)
v3 = v1 + v2  # Usa __add__

print(f"v1: {v1}")     # Usa __str__
print(f"v2: {v2}")      
print(f"v3: v1 + v2 = {v3}")      

print(f"\n¿{v1} == {v3}? : {v1 == v3}") # False
print(f"¿{v1} + {v2} == {v3}? : {v1 + v2 == v3}") # True

v5 = v1 * 3
print(f"\n{v1} * 3 = {v5}")
print(f"¿{v1} < {v2}? : {v1 < v2}")
print(f"Tamaño (dimensión) de {v1}: {len(v1)}")

print(f"\nComponente 0 de {v1}: {v1[0]}")
print(f"Componente 1 de {v1}: {v1[1]}")
v1[0] = 10
print(f"Componente 0 modificada: {v1}")


v1: (2, 3)
v2: (1, 4)
v3: v1 + v2 = (3, 7)

¿(2, 3) == (3, 7)? : False
¿(2, 3) + (1, 4) == (3, 7)? : True

(2, 3) * 3 = (6, 9)
¿(2, 3) < (1, 4)? : True
Tamaño (dimensión) de (2, 3): 2

Componente 0 de (2, 3): 2
Componente 1 de (2, 3): 3
Componente 0 modificada: (10, 3)


### Objetos iterables con `__iter__` y `__next__`

Para que un objeto sea iterable (se pueda usar en un `for`), debe implementar el protocolo iterador:

*   `__iter__(self)`: Debe devolver el objeto iterador. Normalmente se devuelve a sí mismo, `self`.
*   `__next__(self)`: Debe devolver el siguiente elemento o lanzar la excepción `StopIteration` cuando no haya más elementos.

En definitiva, para que un objeto sea un iterador, debe implementar el método `__next__(self)`. Puede ser el mismo objeto, o bien puede ser una instancia de otro tipo de objeto especializado (eso lo decide el método `___iter__(self)`).

Por ejemplo, consideremos una clase Grupo que almacena una lista de personas y que permita iterar sobre sus miembros:

In [2]:
class Persona:
    def __init__(self, nombre):
        self.nombre = nombre
    
    def __str__(self):
        return self.nombre

class Grupo:
    def __init__(self):
        self.__lista = []

    def agregar(self, persona):
        self.__lista.append(persona)

    """ Método que será invocado cuando se ejecute la función incorporada iter() sobre el objeto.
        Inicializa atributos privados que controlan el iterador y se devuelve a sí mismo como objeto iterador.
    """
    def __iter__(self):
        self.__iter_actual = 0
        self.__iter_limite = len(self.__lista)
        return self 

    """ Método que será invocado cuando se ejecute la función incorporada next() sobre el objeto.
        Debe devolver el siguiente elemento de la secuencia. 
        Nota: no confundir con una función generadora!
    """
    def __next__(self):
        if self.__iter_actual < self.__iter_limite:
            valor = self.__lista[self.__iter_actual]
            self.__iter_actual += 1
            return valor
        else:
            raise StopIteration

mi_grupo = Grupo()
mi_grupo.agregar(Persona("Júlia"))
mi_grupo.agregar(Persona("Pau"))
mi_grupo.agregar(Persona("Lídia"))

for i in mi_grupo: # mi_grupo es instancia de la clase Grupo, y es iterable.
    print(i)

Júlia
Pau
Lídia


## Resumen

*   **Métodos Especiales (Dunder Methods)**: Funciones (`__nombre__`) predefinidas en Python para extender el soporte basico de la clase.
*   Permiten dotar de semántica natural al uso de objetos (impresión natural usando `print` con `__str__` o `__repr__`; sumar o calcular magnitudes con `__add__`, etc).
*   **Iterabilidad**: Se consigue proveyendo a los objetos de los métodos `__iter__` y `__next__`, lo cual permite usarlos en bucles for o comprensiones.